In [1]:
import numpy as np
import pandas as pd
import re 

In [2]:
#Create a dataframe for Bandage Combined1
Bandage_Combined1= pd.read_csv('Bandageoutputcombined1.tsv', sep='\t')

In [27]:
Bandage_Combined1.head()

,Query,Path,Length,Query covered by path,Query covered by hits,Mean hit identity,Total hit mismatches,Total hit gap opens,Relative length,Length discrepancy,E-value product,Sequence
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,587,95.915%,95.915%,99.83%,1,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...
1,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593,550,96.4912%,96.4912%,99.455%,3,0,100%,0,0,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...
2,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593,555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
3,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593,555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
4,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,7593,587,95.915%,95.915%,99.659%,2,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...


In [4]:
#Create a dataframe for GraphAligner Combined1
GraphAligner_Combined1=pd.read_csv('GraphAligneroutputcombined1.tsv', sep='\t', names=["Query", "Query Length", "Query Start", 
                                          "Query End","Strand Relative Length","Path Matching","Path Length",
                                         "Start Position on Path","End Position on Path","Number of residues Matches",
                                         "Alignment Back Length","Mapping Quality","Column 1"])

In [28]:
GraphAligner_Combined1.head()


,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,612,25,612,+,7593,2524,55,642,586,587,255,NM:i:1
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,612,2,18,+,1851,79238,37281,37296,15,16,255,NM:i:1
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,570,20,570,+,7593,2524,92,642,547,550,255,NM:i:3
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2


In [29]:
#Format the Query column from  GraphAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


GraphAligner_Combined1['Query'] = GraphAligner_Combined1['Query'].map(format_query)

In [30]:
GraphAligner_Combined1.head()

,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,612,25,612,+,7593,2524,55,642,586,587,255,NM:i:1
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,612,2,18,+,1851,79238,37281,37296,15,16,255,NM:i:1
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,570,20,570,+,7593,2524,92,642,547,550,255,NM:i:3
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2


In [8]:
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('>', "")
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('<', "")

In [31]:
GraphAligner_Combined1.head()

,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,612,25,612,+,7593,2524,55,642,586,587,255,NM:i:1
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,612,2,18,+,1851,79238,37281,37296,15,16,255,NM:i:1
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,570,20,570,+,7593,2524,92,642,547,550,255,NM:i:3
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2


In [10]:
pattern1 = r'\(.*?\)'
def format_path(x):
    return re.sub(pattern1,"", x).rstrip()

In [11]:
Bandage_Combined1['Path']=Bandage_Combined1['Path'].map(format_path)

In [12]:
Bandage_Combined1['Path']=Bandage_Combined1['Path'].str.replace('+', "")
Bandage_Combined1['Path']=Bandage_Combined1['Path'].str.replace('-', "")

In [13]:
Bandage_Combined1

,Query,Path,Length,Query covered by path,Query covered by hits,Mean hit identity,Total hit mismatches,Total hit gap opens,Relative length,Length discrepancy,E-value product,Sequence
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,587,95.915%,95.915%,99.83%,1,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...
1,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593,550,96.4912%,96.4912%,99.455%,3,0,100%,0,0,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...
2,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593,555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
3,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593,555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
4,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,7593,587,95.915%,95.915%,99.659%,2,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...
...,...,...,...,...,...,...,...,...,...,...,...,...
388,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,5967,861,100%,100%,98.955%,9,0,100%,0,0,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
389,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,5967,861,100%,100%,99.768%,2,0,100%,0,0,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
390,gb|AY092401|+|0-861|ARO:3000963|TEM-96,5967,861,100%,100%,99.652%,3,0,100%,0,0,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
391,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1107,1920,100%,100%,97.604%,46,0,100%,0,0,ATGAAAATTATTAATATTGGAGTTTTAGCTCATGTTGATGCAGGAA...


In [14]:
GraphAligner_Combined_Query_Path=GraphAligner_Combined1[['Query','Path Matching']]

In [15]:
GraphAligner_Combined_Query_Path

,Query,Path Matching
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,1851
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593
...,...,...
375,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,5967
376,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,5967
377,gb|AY092401|+|0-861|ARO:3000963|TEM-96,5967
378,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1107


In [16]:
Bandage_Combined_Query_Path=Bandage_Combined1[['Query','Path']]

In [17]:
Bandage_Combined_Query_Path

,Query,Path
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593
1,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593
2,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593
3,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593
4,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,7593
...,...,...
388,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,5967
389,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,5967
390,gb|AY092401|+|0-861|ARO:3000963|TEM-96,5967
391,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1107


In [18]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df=pd.merge(Bandage_Combined_Query_Path,GraphAligner_Combined_Query_Path, on='Query')

In [19]:
df

,Query,Path,Path Matching
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,7593
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,1851
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593,7593
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593,7593
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593,7593
...,...,...,...
383,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,5967,5967
384,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,5967,5967
385,gb|AY092401|+|0-861|ARO:3000963|TEM-96,5967,5967
386,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1107,1107


In [20]:
df.rename(columns = {'Path':'Path_Bandage'}, inplace = True)
df.rename(columns = {'Path Matching':'Path_GraphAligner'}, inplace = True)

In [21]:
df

,Query,Path_Bandage,Path_GraphAligner
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,7593
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,1851
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593,7593
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593,7593
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593,7593
...,...,...,...
383,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,5967,5967
384,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,5967,5967
385,gb|AY092401|+|0-861|ARO:3000963|TEM-96,5967,5967
386,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1107,1107


In [22]:
df['Path_Bandage'] = df['Path_Bandage'].str.strip()
df['Path_GraphAligner'] = df['Path_GraphAligner'].str.strip()

In [23]:
df

,Query,Path_Bandage,Path_GraphAligner
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,7593
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,1851
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593,7593
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593,7593
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593,7593
...,...,...,...
383,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,5967,5967
384,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,5967,5967
385,gb|AY092401|+|0-861|ARO:3000963|TEM-96,5967,5967
386,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1107,1107


In [24]:
#Check for similar path for the queries
df['Equal'] = np.where(df['Path_Bandage']==df['Path_GraphAligner'], 
                                           'yes', 'no')

In [25]:
df

,Query,Path_Bandage,Path_GraphAligner,Equal
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,7593,yes
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,7593,1851,no
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,7593,7593,yes
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,7593,7593,yes
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,7593,7593,yes
...,...,...,...,...
383,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,5967,5967,yes
384,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,5967,5967,yes
385,gb|AY092401|+|0-861|ARO:3000963|TEM-96,5967,5967,yes
386,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1107,1107,yes


In [26]:
df['Equal'].value_counts()

Equal
yes    370
no      18
Name: count, dtype: int64